# Dashboard visualization Otomoto

In [1]:
!pip install dash
!pip install ipython-sql
!pip install psycopg2-binary
!pip install pandas
!pip install numpy
!pip install numpy matplotlib
!pip install folium
!pip install geojson
!pip install jobspy

In [2]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import pylab as pl
import folium
import json
import os
from folium import plugins
%matplotlib inline

import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.express as px

<ipython-input-2-9a3040288960>:15: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-9a3040288960>:16: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
with open('Data_to_Otomoto.json') as file:
    Data_to_Otomoto = json.loads(file.read())
car_brand = Data_to_Otomoto["car_brand"]


In [4]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')

In [5]:
cur = psql.cursor()
sql_otomoto = "SELECT * FROM otomoto_10;"
dat_otomoto = sqlio.read_sql_query(sql_otomoto, psql)
conn = None

In [6]:
car_loc = dat_otomoto[["latitude","longitude","cena","marka_pojazdu"]]

In [7]:
a = 0
for index, row in car_loc.iterrows():
    if (np.isnan(car_loc.at[index,'latitude']) or np.isnan(car_loc.at[index,'longitude']) 
    or np.isnan(car_loc.at[index,'cena'])):
        car_loc = car_loc.drop([index])
        a += 1
print("Drop rows where Nan from table otomoto: ", a)


Drop rows where Nan from table otomoto:  2218


In [8]:
car_loc = car_loc.reset_index(drop=True)

In [9]:
# for brand in car_loc:
    

In [10]:
def world_new():    
    my_world = folium.Map(
    zoom_start=6,
    location=[51.9194, 19.1451], prefer_canvas=True)
    my_world = plugins.MarkerCluster().add_to(my_world)
    return my_world

In [11]:
#     my_world = world_new()

#     for row in range(len(car_loc.index)):
#         folium.CircleMarker(
#             location=[car_loc.at[row,'latitude'], car_loc.at[row,'longitude']],
#             radius=3,
# #             popup='Sell price: ' + str(lok_sell_limit.at[row,'price']),
#             color='red',
#             fill=True,
#             fill_color='red',
#             fill_opacity=1
#         ).add_to(my_world)
    
#     html_string = my_world.get_root().render()

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div(style={'text-align': 'center'}, children=[
    html.H1('Cars for sell in Otomoto',style={'textAlign': 'center','front-size' :50}),
    html.Br(),
    dcc.Input(id="Price_MIN", type="number", placeholder="Price_MIN", value=1000),
    dcc.Input(id="Price_MAX", type="number", placeholder="Price_MAX", value=1000000),
    html.Button(id='my-button', n_clicks=0, children="Update"),
    html.Br(),
    html.Iframe(id = 'map', srcDoc = world_new().get_root().render(),
                width = '700',height = '700')
    ])
@app.callback(
    [dash.dependencies.Output('map', 'srcDoc')],
    [dash.dependencies.State('Price_MIN', 'value'),
     dash.dependencies.State('Price_MAX', 'value')],
    [dash.dependencies.Input('my-button', 'n_clicks')]
    )
    
def Rent_Price_Limiter(Price_MIN, Price_MAX, n_clicks):
    my_world = world_new()
    
    car_loc_limit = car_loc[car_loc.cena.between(Price_MIN, Price_MAX, inclusive=False)]
    car_loc_limit = car_loc_limit.reset_index(drop=True)
    
    for row in range(len(car_loc_limit.index)):
        folium.CircleMarker(
            location=[car_loc_limit.at[row,'latitude'], car_loc_limit.at[row,'longitude']],
            radius=3,
            popup='Price: ' + str(car_loc_limit.at[row,'cena']) + '<br>' +str(car_loc_limit.at[row,'marka_pojazdu']) ,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1
        ).add_to(my_world)
    print(n_clicks)
    html_string = my_world.get_root().render()
    
    return (html_string,)

if __name__ == '__main__':
    app.run_server(host='0.0.0.0')

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.17.0.2:8050/ (Press CTRL+C to quit)
192.168.10.24 - - [27/Sep/2021 21:01:50] "GET / HTTP/1.1" 200 -
192.168.10.24 - - [27/Sep/2021 21:01:51] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.10.24 - - [27/Sep/2021 21:01:51] "GET /_dash-layout HTTP/1.1" 200 -
192.168.10.24 - - [27/Sep/2021 21:01:51] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
192.168.10.24 - - [27/Sep/2021 21:01:51] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -


0


192.168.10.24 - - [27/Sep/2021 21:02:10] "POST /_dash-update-component HTTP/1.1" 200 -


1


192.168.10.24 - - [27/Sep/2021 21:02:43] "POST /_dash-update-component HTTP/1.1" 200 -


2


192.168.10.24 - - [27/Sep/2021 21:03:11] "POST /_dash-update-component HTTP/1.1" 200 -
